In [1]:
import os
import sys
import argparse
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [3]:
from nnest.trainer import Trainer

In [4]:
dims = 2

In [5]:
test_data = np.random.normal(size=(10, dims))
test_data = torch.from_numpy(test_data).float()

In [6]:
trainers = []
trainers.append(Trainer(dims, 64,  num_blocks=2, num_layers=1, flow='nvp'))
trainers.append(Trainer(dims, 64,  flow='spline'))

[nnest.trainer] [INFO] SingleSpeedNVP(
  (flow): NormalizingFlow(
    (flows): ModuleList(
      (0): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
          (4): Linear(in_features=64, out_features=2, bias=True)
        )
        (translate_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): ReLU()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): ReLU()
          (4): Linear(in_features=64, out_features=2, bias=True)
        )
      )
      (1): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
          (4): Linear(in_features=64, out_features=2, bias=True)
      

In [7]:
for t in trainers:
    z, log_det = t.netG.forward(test_data)
    print(z.shape, log_det.shape)
    x, log_det = t.netG.inverse(z)
    print(z.shape, log_det.shape)
    print(torch.max(x - test_data))
    log_probs = t.netG.log_probs(test_data)
    print(log_probs.shape)
    samples = t.netG.sample(10)
    print(samples.shape)

torch.Size([10, 2]) torch.Size([10])
torch.Size([10, 2]) torch.Size([10])
tensor(1.1921e-07, grad_fn=<MaxBackward1>)
torch.Size([10])
torch.Size([10, 2])
torch.Size([10, 2]) torch.Size([10])
torch.Size([10, 2]) torch.Size([10])
tensor(1.3337e-06, grad_fn=<MaxBackward1>)
torch.Size([10])
torch.Size([10, 2])


In [8]:
num_slow = 2
num_fast = 2
dims = num_slow + num_fast

In [9]:
test_data = np.random.normal(size=(10, dims))
test_data = torch.from_numpy(test_data).float()

In [10]:
trainers = []
trainers.append(Trainer(dims, 64,  num_slow=num_slow, num_blocks=1, num_layers=1, flow='nvp'))
trainers.append(Trainer(dims, 64,  num_slow=num_slow, flow='spline'))

[nnest.trainer] [INFO] FastSlowNVP(
  (slow_flow): NormalizingFlow(
    (flows): ModuleList(
      (0): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
          (4): Linear(in_features=64, out_features=2, bias=True)
        )
        (translate_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): ReLU()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): ReLU()
          (4): Linear(in_features=64, out_features=2, bias=True)
        )
      )
    )
  )
  (fast_flow): NormalizingFlow(
    (flows): ModuleList(
      (0): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
  

[nnest.trainer] [INFO] Number of network params: [21168]
[nnest.trainer] [INFO] Device [cpu]


In [15]:
for t in trainers:
    z, log_det = t.netG.forward(test_data)
    print(z.shape, log_det.shape)
    x, log_det = t.netG.inverse(z)
    print(z.shape, log_det.shape)
    print(torch.max(x - test_data))
    log_probs = t.netG.log_probs(test_data)
    print(log_probs.shape)
    samples = t.netG.sample(10)
    print(samples.shape)
    dz = torch.randn_like(z) * 0.01
    dz[:, 0:num_slow] = 0.0
    xp, log_det = t.netG.inverse(z+dz)
    print(torch.max((x-xp)[:, 0:num_slow])) 

torch.Size([10, 4]) torch.Size([10])
torch.Size([10, 4]) torch.Size([10])
tensor(1.1921e-07, grad_fn=<MaxBackward1>)
torch.Size([10])
torch.Size([10, 4])
tensor(0., grad_fn=<MaxBackward1>)
torch.Size([10, 4]) torch.Size([10])
torch.Size([10, 4]) torch.Size([10])
tensor(9.5367e-07, grad_fn=<MaxBackward1>)
torch.Size([10])
torch.Size([10, 4])
tensor(0., grad_fn=<MaxBackward1>)
